# 뉴스들을 정리해서 보여주기

### 네이버 뉴스 페이지

<img src="naver_news_example.png" alt="Drawing" style="width: 500px;"/>


다음과 같은 페이지의 로직을 한번 만들어 보겠습니다.

필요한 로직은 2가지 입니다.


### 1. Grouping

위의 경우는 세가지 큰 주제와 그에따른 기사들로 그룹이 설정되어있습니다.

        1. 롯데 후계자는 신동빈 / 신격호 유언장에
        
        2. SK바이오팜 / 청약경쟁률 323대 1
        
        3. 쿠팡 덕평 물류센터 / 코로나 19 확진자 발생으로 폐쇄
       
수백, 수천개의 기사들을 각각의 토픽에 맞게 그룹을 하는 방식이 필요합니다


### 2. Keyword Extraction

각각의 기사를 그룹을 하였으면, 그룹에서 키워드를 뽑아서 주제를 설정하는것이 필요합니다.

# 코드를 만들어보자

In [1]:
from konlpy.tag import Mecab
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from gensim import corpora
import gensim
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib
from pandas.plotting import register_matplotlib_converters
import networkx as nx
from bs4 import BeautifulSoup
import requests

## 시각화를 위한 library
import matplotlib.pyplot as plt
import seaborn as sns

import text_helpers as t_helper

from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

mecab = Mecab()

sns.set()
register_matplotlib_converters()
plt.rcParams["font.family"] = 'AppleGothic'
plt.rcParams["font.size"] = 20
plt.rcParams["figure.figsize"] = (14,4)
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
news_db = pd.read_csv('news_title_fixed.csv')

stock_info_path = "stock_info_0615.csv"
stock_data = pd.read_csv(stock_info_path, index_col='Unnamed: 0')

news_db_test = news_db[(news_db['PUB_DT']>='2020-05-01') & (news_db['PUB_DT']<'2020-05-06')].reset_index(drop=True)

## 저희가 사용할 기사는 5월 1일부터 5월 5일까지 의 기사들입니다

1000개가 조금 안되는 기사입니다.

이 기사들을 주요 토픽별로 묶는 알고리즘을 우선 보여드리겠습니다.

In [3]:
news_db_test

,PUB_DT,GICODE,TITLE,BODY,URL,PROVIDER,company_name
0,2020-05-01 01:48:00,A004170,정용진 신세계 부회장 저택 149억원… 경기도서 가장 비싸,NaN,http://finance.naver.com/item/news_read.nhn?ar...,조선비즈,"('신세계',)"
1,2020-05-01 05:36:00,A005490,"조직 협업문화 확산 나선 포스코 “업무지식, 포스위키에 물어보세요”",NaN,http://finance.naver.com/item/news_read.nhn?ar...,이데일리,"('POSCO',)"
2,2020-05-01 06:01:00,A001510,코로나19 보복성 소비로 의류·음식료 수혜 전망 SK증권,NaN,http://finance.naver.com/item/news_read.nhn?ar...,파이낸셜뉴스,"('SK증권',)"
3,2020-05-01 06:01:00,A006400,"삼성SDI, 2분기까지 보릿고개..하반기엔 이익 개선 IBK투자증권",NaN,http://finance.naver.com/item/news_read.nhn?ar...,파이낸셜뉴스,"('삼성SDI',)"
4,2020-05-01 07:01:00,A005380,"현대차, 영국서 첼시FC와 축구 홈트레이닝 영상제작",NaN,http://finance.naver.com/item/news_read.nhn?ar...,파이낸셜뉴스,"('현대차',)"
...,...,...,...,...,...,...,...
967,2020-05-05 18:07:00,A042670,"두산인프라코어, 中서 대형 굴착기 20대 연이어 수주",NaN,http://finance.naver.com/item/news_read.nhn?ar...,파이낸셜뉴스,"('두산인프라코어',)"
968,2020-05-05 18:11:00,A032640,LG유플러스 5G 단독모드 상용화 검증 완료,NaN,http://finance.naver.com/item/news_read.nhn?ar...,파이낸셜뉴스,"('LG유플러스',)"
969,2020-05-05 18:22:00,A139480,가을 새우는 옛말… 이마트 ‘국산 생새우’ 연중 판매,NaN,http://finance.naver.com/item/news_read.nhn?ar...,파이낸셜뉴스,"('이마트',)"
970,2020-05-05 18:22:00,A282330,"BGF리테일 ""희망의 도토리나무 키워요""",NaN,http://finance.naver.com/item/news_read.nhn?ar...,파이낸셜뉴스,"('BGF리테일',)"


우선 사용하는 로직은 기사의 제목만으로 분류를 하는 것입니다.

In [4]:
new_res, word_dict, tfidf_data = t_helper.theme_analysis(news_db_test, 10, 200, print_key=False)

이렇게 분류 후 972개의 기사가 총 50개의 토픽으로 분류가 되었습니다.


### 주의 : 알고리즘에 Randomness 가 조금씩 섞여있어서, 토픽의 숫자 / 인덱싱이 매번 돌릴때마다 조금씩 바뀝니다.



In [5]:
#### TEST APPLY 용 : Randomness check #####
for topic_num in new_res['micro_topic'].unique():
    print(topic_num)
    print(new_res[new_res['micro_topic']==topic_num]['TITLE'])
    print('\n\n')

0.0
0                정용진 신세계 부회장 저택 149억원… 경기도서 가장 비싸
8              최태원 회장 “SK넥실리스, 명실상부한 SK 일원된 것 환영”
9          진매트릭스, '네오플렉스 RV-Panel A' 식약처 품목 허가 획득
24     PC방창업 브랜드 아이센스리그PC방, 각 부서 전문가들이 직접 창업컨설...
29               한기평 "농협생명, 보험지급능력평가 AAA→AA+로 강등"
                          ...                    
953       GS건설, `DMC리버파크자이·DMC리버포레자이` 특별공급 성공적 마감
960                SK이노베이션 美법인, 휘발유 가격담합 혐의로 기소당해
963                      작년 CJ대한통운 택배 13억건 분석해보니…
968                      LG유플러스 5G 단독모드 상용화 검증 완료
970                        BGF리테일 "희망의 도토리나무 키워요"
Name: TITLE, Length: 160, dtype: object



27.0
1      조직 협업문화 확산 나선 포스코 “업무지식, 포스위키에 물어보세요”
62                  지하배관 안뜯어도 한눈에…포스코 신기술 눈길
889               버리던 슬러지, 협력사 손잡고 재활용하는 포스코
908        포스코, 스마트팩토리 설립 적극 지원…'기업시민' 역할 톡톡
910              [현장에서]'기업시민' 등진 포스코의 물류업 진출
915            포스코·현대제철, 수요 부진·원자재값 상승 '이중고'
916            포스코·현대제철, 수요 부진·원자재값 상승 '이중고'
964         포스코, 지난해 환경투자 '역대 최대'…총 투자비의 27%
Name: TITLE, dtype: object



3.0
2   

몇몇 그룹의 경우 단일 기사 내용으로 모아지는 경우가 있지만, 대부분의 경우, 기사의 숫자가 많기 때문에, 여러 내용의 기사가 한 그룹으로 섞이는 경우가 있습니다.

아래 예시를 볼까요 ?

In [17]:
topic_num = 16

selected_news = new_res[new_res['micro_topic']==topic_num]


###### TF-IDF 어레이를 통해서 상위 점수를 받은 단어들을 추출 #####

index_row = np.array([np.linspace(0,len(word_dict)-1,len(word_dict),dtype='int')])
score_row = np.array([np.sum(tfidf_data[selected_news.index.values],axis=0)])
index_score_format = np.concatenate((index_row, score_row)).T

index_score_format_sort = index_score_format[index_score_format[:,1].argsort()][::-1]

## 점수별로 sorting 후, 상위 단어 추출
up_to = 5

word_row = np.array(word_dict)[index_score_format_sort[:,0][:up_to].astype(int)]

word_score_format = np.concatenate((np.array([word_row]), np.array([index_score_format_sort[:,1][:up_to]]))).T


그중 한 토픽을 살펴보면 다음과 같습니다

In [18]:
### 출력하기 ####
print(topic_num)
for title in selected_news['TITLE'].values:
    print(title)

print('\n')
print('키워드')
for word, score in word_score_format:
    print('%s  : %.3f'%(word, float(score)))

16
삼성전자 ‘반도체 비전 2030’ 1년···1위 등극 쉽지 않네
삼성전자, 맞춤 디자인 비스포크…색상 10가지 확장한 이유
삼성전자, 'CES 최고혁신상' 2020년형 사운드바 국내 출시
삼성전자, CES2020 `최고혁신상` 수상한 사운드바 국내 출시
삼성전자, 2020년형 사운드바 국내 출시
삼성전자, 'CES 최고혁신상' 사운드바 국내 출시
삼성전자, 2020년형 사운드바 국내 출시...고품격 음질 구현
삼성전자, 'CES 최고혁신상' 수상 사운드바 국내 출시
삼성전자, `갤럭시Z 플립` 미러 골드 6일 국내 출시
법인세 낮춰 `기업 족쇄` 풀자 삼성전자 이익은 MS의 절반
삼성전자 법인세율 2년새 3.7%P 뛸때 경쟁사 애플은 9.4%P 세율...
'CES 최고 혁신상' 수상…삼성전자 사운드바 출시
삼성전자 '신개념' 정수기 출시 임박…비스포크 흥행 잇나
올 여름 무더위 예고, 삼성전자 무풍에어컨 사전점검으로 대비하세요!
반려동물 털 걱정 끝 삼성전자, 펫케어 공기청정기 출시
반려동물의 냄새부터 털까지 …삼성전자, 펫케어 공기청정기 출시
삼성전자, 반려동물 전용 공기청정기 출시…“털과 냄새 말끔히 제거”
삼성전자, 펫팸족 겨냥 '펫케어 공기청정기' 출시
삼성전자, '펫케어 공기청정기' 출시
횡보하는 삼성전자 주가…증권가 전망도 엇갈려
삼성전자, 브라질에 12억원대 의료용품·전자제품 지원
'어린이 펀드' 뭐 담았나 봤더니…`삼성전자`
우리 아이에 줄 주식, 테슬라가 좋을까 삼성전자가 좋을까
삼성전자, 1구 인덕션 '더 플레이트' 출시…"혼밥에 딱"
삼성전자, 포터블 인덕션 '더 플레이트' 1구 모델 6일 출시
삼성전자, 포터블 인덕션 ‘더 플레이트’ 신제품 출시
삼성전자 인덕션 시장 공략… 포터블 인덕션 ‘더 플레이트’ 신제품 출...
삼성전자, 포터블 인덕션 ‘더 플레이트’ 신제품 출시
삼성전자, 포터블 인덕션 1구형 출시...."혼밥에 딱"
삼성전자, 브라질에 12억원대 의료용품·전자제품 지원


키워드
삼성전자  : 17.922
출시  : 9.

다음을 통해 알 수 있는 사실은, 삼성전자 / 출시 / 사운드 등으로 현재의 토픽이 그룹핑이 된것입니다.

뭔가 조금은 아쉬운것을 느낄 수 있습니다.

여기서 이제 한번 더, 작업을 통해서 제대로 된 토픽 그룹핑을 실시합니다.

### 사용로직 

    1. 기사를 추출해서 기사의 문장을베이스로 단워 네트워크를 구축 합니다. (PageRank 알고리즘)

    단어 네트워크를 통해서 각 기사들을 비교, 같은 내용을 포함하는지 확인합니다.

## 완벽한 세분화를 위해서는 기사를 로딩해야합니다 ##

In [19]:
articles = []

for i, row in selected_news.iterrows():
    article = t_helper.get_article(row)
    articles.append(article)

/Users/jayden.jeon/opt/anaconda2/envs/quant3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Users/jayden.jeon/opt/anaconda2/envs/quant3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Users/jayden.jeon/opt/anaconda2/envs/quant3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Users/jayden.jeon/

/Users/jayden.jeon/opt/anaconda2/envs/quant3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Users/jayden.jeon/opt/anaconda2/envs/quant3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Users/jayden.jeon/opt/anaconda2/envs/quant3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/Users/jayden.jeon/

## 각각의 기사들을 로딩 한 후, 단어 네트워크를 생성, 비교합니다 ##

In [20]:
clustering_map = t_helper.create_cluster_map(articles)

## 여기서 조금 더 나아가서, 각 기사그룹마다, 가장 핵심 키워드와, 핵심 기사 본문을 추출합니다.



In [26]:
from textrank import KeywordSummarizer
from textrank import KeysentenceSummarizer


for i, cluster in enumerate(clustering_map):
    articles_index = clustering_map[cluster]
    for index in articles_index:
        print(selected_news['TITLE'].values[index])
        
        
        
    # Article 들을 전부 다 뭉치게 만든다.
    cluster_articles = np.array(articles)[articles_index]
    
    ## 기사를 각 문장을 element 로 취급하는 list 로 만들기
    article_sentences_list = []
    ## 기사 요약에 필요
    sentence_info = []
    
    for article in cluster_articles:
        for sentence in article:
            article_sentences_list.append(sentence)
        sentence_info.append(len(article))

            
    ### 키워드 추출 ###
    keyword_extractor = KeywordSummarizer(tokenize = t_helper.mecab_tokenizer, window = -1)
    keywords = keyword_extractor.summarize(article_sentences_list, topk=30)
    print('\n')
    print('키워드 : ')
    keyword_list = []
    for keyword, _ in keywords[:5]:
        keyword_list.append(keyword)
    print(keyword_list)
    

    key_sentences = t_helper.get_key_sentences(article_sentences_list, 5, sentence_info)
    
    print('\n')
    ### PRINT ###
    print('기사의 주 내용 : ')
    for sentence in key_sentences:
        print(sentence)

    print('\n\n')


삼성전자 ‘반도체 비전 2030’ 1년···1위 등극 쉽지 않네


키워드 : 
['삼성전자', '이미지센서', '지난해', '점유', 'AP']


기사의 주 내용 : 
 TSMC의 1·4분기 점유율은 지난해 같은 기간보다 6%포인트 늘어난 반면 삼성전자의 점유율은 3
 삼성전자가 올해 출시한 플래그십 스마트폰 갤럭시S20의 국내용 제품에 자사 AP인 ‘엑시노스’ 대신 퀄컴의 ‘스냅드래곤 865’를 채택한 게 대표적이다
 이미지센서 분야에서는 부동의 1위 소니와 점유율 격차는 크지만 삼성전자가 기술력에서는 앞서 있다는 평가를 받는다
 다만 삼성전자의 이미지센서 시장 점유율은 소니에 크게 뒤져 있다
 시장조사업체 테크노시스템리서치에 따르면 지난해 기준 삼성전자는 이미지센서 시장에서 17



삼성전자, 맞춤 디자인 비스포크…색상 10가지 확장한 이유


키워드 : 
['색상', '냉장고', '삼성전자', '비스포크', '도시']


기사의 주 내용 : 
삼성전자, 맞춤 디자인 비스포크…색상 10가지 확장한 이유
[아시아경제 이동우 기자] 삼성전자는 맞춤형 냉장고 비스포크의 신규 색상을 세계 여러 도시에서 영감을 얻은 10종을 도입해, 총 15종으로 강화했다
삼성전자는 이번 비스포크 신규 색상은 스웨덴의 스톡홀름과 독일의 베를린, 대한민국 서울의 색채와 건축물 등에서 영감을 받았다
 딥그린과 버건디 색상은 베를린이라는 도시의 폭넓은 디자인적 스펙트럼을 상징한다
앞서 삼성전자는 지난달 새로워진 비스포크 냉장고 4도어 4개 모델의 사전 판매를 시작한 바 있다



삼성전자, 'CES 최고혁신상' 2020년형 사운드바 국내 출시
삼성전자, CES2020 `최고혁신상` 수상한 사운드바 국내 출시
삼성전자, 2020년형 사운드바 국내 출시
삼성전자, 'CES 최고혁신상' 사운드바 국내 출시
삼성전자, 2020년형 사운드바 국내 출시...고품격 음질 구현
삼성전자, 'CES 최고혁신상' 수상 사운드바 국내 출시
'CES 최고 혁신상' 수상…삼성전자 사운드바 출시
삼성전자,

기사 클러스터링의 경우, 거의 완벽한것을 볼 수 있습니다.

하지만 핵심 기사를 뽑는 경우, 조금 부족한 면이 있어서, 여러모로 보완이 필요해보입니다.